In [3]:
import ibis
import duckdb

SCALE_FACTOR = 1

# Connect to DuckDB
duckdb_con = duckdb.connect("tpch.duckdb")
con = ibis.connect("duckdb://tpch.duckdb")

In [4]:
# Generate TPC-H data at scale factor 1 (1 GB)
duckdb_con.sql(f"CALL dbgen(sf={SCALE_FACTOR})")

# Export each table to Parquet format
tables = ["lineitem", "orders", "customer", "supplier", "nation", "region", "part", "partsupp"]
for table in tables:
    duckdb_con.sql(f"COPY {table} TO 'data/{table}.parquet' (FORMAT PARQUET)")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

IOException: IO Error: Cannot open file "data/lineitem.parquet": No such file or directory